In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import os
from torch.utils.data import DataLoader
from PIL import Image

In [27]:
source_root = '../../data/classification_direction'
train_root = '../../data/classification_direction/train'
val_root = '../../data/classification_direction/val'

In [28]:
# import os
# import random
# from shutil import copy2
# from torchvision.datasets import ImageFolder

# def create_dir_structure(root_path, classes):
#     for cls in classes:
#         os.makedirs(os.path.join(root_path, cls), exist_ok=True)

# def copy_files(file_paths, source_root, target_root):
#     for file_path in file_paths:
#         class_name = os.path.basename(os.path.dirname(file_path))
#         target_path = os.path.join(target_root, class_name, os.path.basename(file_path))
#         copy2(file_path, target_path)

# # Загрузка исходного датасета
# dataset = ImageFolder(root=source_root)
# class_names = dataset.classes

# # Определим количество данных для тренировки и валидации
# train_ratio = 0.8
# total_size = len(dataset)
# train_size = int(total_size * train_ratio)
# val_size = total_size - train_size

# # Перемешаем данные и разделим их
# indices = list(range(total_size))
# random.shuffle(indices)
# train_indices, val_indices = indices[:train_size], indices[train_size:]

# train_files = [dataset.imgs[idx][0] for idx in train_indices]
# val_files = [dataset.imgs[idx][0] for idx in val_indices]

# # Создадим структуру папок для тренировочных и валидационных данных
# create_dir_structure(train_root, class_names)
# create_dir_structure(val_root, class_names)

# # Скопируем файлы в соответствующие папки
# copy_files(train_files, source_root, train_root)
# copy_files(val_files, source_root, val_root)


In [29]:
# Гиперпараметры
batch_size = 16
num_epochs = 8
learning_rate = 0.0005

In [30]:
# Трансформации для данных
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [31]:
# Загрузка данных
train_dataset = datasets.ImageFolder(root=train_root, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [32]:
val_dataset = datasets.ImageFolder(root=val_root, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [33]:
# Загрузка модели MobileNet
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)  # Заменяем последний слой для бинарной классификации

In [34]:
# Использование GPU, если доступно
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [35]:
# Функция потерь и оптимизатор
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [36]:
# Обучение модели
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float()
        
        # Обнуление градиентов
        optimizer.zero_grad()
        
        # Прямой проход
        outputs = model(inputs)
        outputs = outputs.squeeze()
        loss = criterion(outputs, labels)
        
        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    # Оценка модели на валидационном наборе
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            outputs = model(inputs)
            outputs = outputs.squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            
            predicted = torch.sigmoid(outputs).round()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / total
    print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')

print('Обучение завершено')


Epoch 1/8, Loss: 0.5531
Validation Loss: 0.2681, Accuracy: 0.9333
Epoch 2/8, Loss: 0.0826
Validation Loss: 0.0867, Accuracy: 0.9333
Epoch 3/8, Loss: 0.0572
Validation Loss: 0.0669, Accuracy: 0.9333
Epoch 4/8, Loss: 0.1482
Validation Loss: 0.0147, Accuracy: 1.0000
Epoch 5/8, Loss: 0.0133
Validation Loss: 0.0203, Accuracy: 1.0000
Epoch 6/8, Loss: 0.0038
Validation Loss: 0.0272, Accuracy: 1.0000
Epoch 7/8, Loss: 0.0136
Validation Loss: 0.0218, Accuracy: 1.0000
Epoch 8/8, Loss: 0.0156
Validation Loss: 0.0092, Accuracy: 1.0000
Обучение завершено


In [40]:
import onnx

# Экспорт в формат ONNX
dummy_input = torch.randn(1, 3, 224, 224).to(device)
torch.onnx.export(model, dummy_input, "mobilenet_v2.onnx", input_names=["input"], output_names=["output"], opset_version=11)

print("Модель сохранена и экспортирована в формат ONNX")

Модель сохранена и экспортирована в формат ONNX


In [49]:
import onnxruntime as ort
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

class ImageClassifierONNX:
    def __init__(self, onnx_path):
        self.onnx_session = ort.InferenceSession(onnx_path)
        self.input_name = self.onnx_session.get_inputs()[0].name
        self.output_name = self.onnx_session.get_outputs()[0].name
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def predict(self, image_path):
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image).unsqueeze(0).numpy()
        outputs = self.onnx_session.run([self.output_name], {self.input_name: image})
        output = outputs[0][0][0]
        return output


In [56]:
classifier_onnx = ImageClassifierONNX('mobilenet_v2.onnx')
for path in os.listdir('../../data/classification_direction/val/bad/'):
  result_onnx = classifier_onnx.predict(f'../../data/classification_direction/val/bad/{path}')
  print(f'Результат классификации: {result_onnx:.4f}')

print()
for path in os.listdir('../../data/classification_direction/val/good/'):
  result_onnx = classifier_onnx.predict(f'../../data/classification_direction/val/good/{path}')
  print(f'Результат классификации: {result_onnx:.4f}')

Результат классификации: -11.2225
Результат классификации: -7.0359
Результат классификации: -9.4938
Результат классификации: -12.2561

Результат классификации: 3.7085
Результат классификации: 2.2994
Результат классификации: 5.9101
Результат классификации: 11.5769
Результат классификации: 9.5283
Результат классификации: 11.6647
Результат классификации: 7.7167
Результат классификации: 9.1278
Результат классификации: 4.3042
Результат классификации: 7.9933
Результат классификации: 9.6085
